-----
# cartoGRAPHs - Figure 2C
-----

In [1]:
#from cartoGRAPHs import * 

%cd pyfiles 
from cartoGRAPHs import *

/Users/chris/Desktop/Github/cartoGRAPHs/pyfiles


In [2]:
%cd ..

/Users/chris/Desktop/Github/cartoGRAPHs


_____
# HUMAN PPI NETWORK
_____

In [3]:
organism = 'human'
G = load_graph(organism)

d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

# Human specific > to include Disease Ontology
if organism == 'human':
    d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
    d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
    d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
    d_names_do = {y:x for x,y in d_do_names.items()}
else: 
    pass

netlayout = 'global'
DM = load_datamatrix(G,organism, netlayout)

In [4]:
G.name = "Human Interactome"
G.graph["desc"] = "The human interactome is a complex, dynamic network of protein-protein interactions that are essential for many biological processes. It is estimated that the human interactome consists of hundreds of thousands of interactions between thousands of proteins. These interactions can take many forms, including direct physical binding, allosteric regulation, and post-translational modifications. Many proteins participate in multiple interactions, forming highly interconnected clusters or modules within the network. The human interactome is not static and can change in response to cellular and environmental cues. "

_____ 
## SPATIAL EMBEDDING
_____

### 2D PORTRAIT

In [5]:
%%time 

posG_umap2D = layout_portrait_umap(G,DM,2,n_neighbors=12, spread=1, min_dist=0.2, metric='cosine')  #layout_portrait_umap(G,DM,2,n_neighbors=8, spread=6, min_dist=0.9, metric='cosine') 
posG = posG_umap2D

CPU times: user 3min, sys: 17 s, total: 3min 17s
Wall time: 2min 25s


____________
## VISUAL SETTINGS + PLOT PREPARATION
____________

### NODES GENERAL 

In [6]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 0.01

scale_factor3D = 0.025
size = list(draw_node_degree_3D(G, scale_factor3D).values())

edge_color = '#ACACAC'
width_edges = 0.15
opacity_edges = 0.1

---
## TOPOGRAPHIC MAP 
---

In [7]:
z_feat = 'disease'
DM_feature  = pd.read_pickle('input/Features_Disease_Dataframe_Human.pickle')

# set gene list (= G.nodes())
genes_assoc = []
for i in DM_feature.index:
    genes_assoc.append(str(i))

genes_notassoc = [] 
for g in G.nodes():
    if g not in genes_assoc:
        genes_notassoc.append(g)

features_counted = (DM_feature == 1).astype(int).sum(axis=1)
d_features_counted = dict(zip(genes_assoc, list(features_counted)))
    
d_rest = {}
for i in genes_notassoc: 
    d_rest[i] = -20
    
d_param_unsorted = {**d_features_counted, **d_rest}
d_z = {key:d_param_unsorted[key] for key in G.nodes()}
    
d_clos = {}
for k,v in d_centralities.items():
    d_clos[k] = v[1]

d_nodecolors = d_clos 
col_pal = 'YlOrRd'

d_colors = color_nodes_from_dict(G, d_nodecolors, palette = col_pal)
colors = list(d_colors.values())

posG3D = layout_topographic(posG, d_z)

---
## PLOT 
---

In [8]:
umap_nodes = get_trace_nodes_3D(posG3D, l_features, colors, size, opacity_nodes)
umap_edges = get_trace_edges_3D(G, posG3D, edge_color, opac = 0.05, linewidth = 0.5)
data = [umap_edges, umap_nodes]

path = 'output/'
schema = 'light' #'dark'
fname = 'FIG2C-topographic_z_'+z_feat+'_'+netlayout+'_'+organism+'_'+schema

plot_3D(data,path,fname, schema)

'output/FIG2C-topographic_z_disease_global_human_light.html'

### 

---
## VRNETZER2 (multiplayer version) EXPORT
---

In [9]:
# annotations 

d_gene_donames = {}
for node, doid in d_gene_do.items():
    sub=[]
    for do,name in d_do_names.items():
        if str(do) in doid:
            sub.append(name)
        d_gene_donames[node] = sub

d_allgene_donames = {}
for g in G.nodes(): 
    if g in d_gene_donames.keys():
        d_allgene_donames[g] = d_gene_donames[g]
    else:
        d_allgene_donames[g] = 'No Disease associated.'

# node annotations
l_annotations_csv = ['Gene: '+str(i)+'; ID:  '+str(j)+'; Diseases: '+str(k) for i,j,k in zip(d_gene_sym.values(), d_gene_sym.keys(),d_allgene_donames.values())]
l_annotations_json = [list(("Gene: "+str(i),"ID: "+str(j),"Diseases: "+str(k))) for i,j,k in zip(d_gene_sym.values(), d_gene_sym.keys(), d_allgene_donames.values())]
d_annotations_csv = dict(zip(list(G.nodes()),l_annotations_csv))
d_annotations_json = dict(zip(list(G.nodes()),l_annotations_json))

In [10]:
# link colors 

# only show links between disease assoc. proteins (i.e. z != 0) 
d_links_colors = {}
for link in G.edges():
    if link[0] in genes_assoc and link[1] in genes_assoc:
        d_links_colors[link] = '#696969'
    else:
        d_links_colors[link] = (0,0,0,0)


In [11]:
# no clusters
clusterlist = ["", 0]
dict_for_cluster = {}

In [12]:
# ---------------
# 2D
#----------------
filename = 'VR/temp/'+'2C-'+G.name+'-topographic2D'
exportVRNetzer_CSV(filename, G, posG, d_colors, d_annotations_csv, d_links_colors, clusterlist)
exportVRNetzer_JSON(filename, G, posG, dict_for_cluster, d_colors, d_annotations_json, d_links_colors)

# ---------------
# 3D
#----------------  
filename = 'VR/temp/'+'2C-'+G.name+'-topographic3D'
exportVRNetzer_CSV(filename, G, posG3D, d_colors, d_annotations_csv, d_links_colors, clusterlist)
exportVRNetzer_JSON(filename, G, posG3D, dict_for_cluster, d_colors, d_annotations_json, d_links_colors)


Exported Files: 
 ['VR/temp/2C-HumanInteractome-topographic2D_nodepositions.csv', 'VR/temp/2C-HumanInteractome-topographic2D_nodecolors.csv', 'VR/temp/2C-HumanInteractome-topographic2D_nodeproperties.csv', 'VR/temp/2C-HumanInteractome-topographic2D_links.csv', 'VR/temp/2C-HumanInteractome-topographic2D_linkcolors.csv', 'VR/temp/2C-HumanInteractome-topographic2D_clusterlabels.csv']
Exported File: 
 ['VR/temp/2C-HumanInteractome-topographic2D.json']
Exported Files: 
 ['VR/temp/2C-HumanInteractome-topographic3D_nodepositions.csv', 'VR/temp/2C-HumanInteractome-topographic3D_nodecolors.csv', 'VR/temp/2C-HumanInteractome-topographic3D_nodeproperties.csv', 'VR/temp/2C-HumanInteractome-topographic3D_links.csv', 'VR/temp/2C-HumanInteractome-topographic3D_linkcolors.csv', 'VR/temp/2C-HumanInteractome-topographic3D_clusterlabels.csv']
Exported File: 
 ['VR/temp/2C-HumanInteractome-topographic3D.json']
